In [ ]:
import os
ROOT_DIR = "/home/ebr/projects/inundation-emulator"
os.chdir(ROOT_DIR)

In [ ]:
from src.emulator import Emulator, DataReader
import tensorflow as tf
import matplotlib.pyplot as plt

In [ ]:
# Load pretrained emulator.

GENERATED_DIR = "/home/ebr/projects/inundation-emulator/generated"
TOPO_FILE = '/home/ebr/data/PTHA2020_runs_UMA/Catania/C_CT.grd'
TOPO_MASK = '/home/ebr/data/PTHA2020_runs_UMA/Catania/ct_mask.txt'
TRAIN_SCENARIOS ="/home/ebr/data/PTHA2020_runs_UMA/train_591/scenarios.txt"
TRAIN_DIR = '/home/ebr/data/PTHA2020_runs_UMA/train_591'
VALIDATION_SCENARIOS = '/home/ebr/data/PTHA2020_runs_UMA/test/scenarios.txt'
VALIDATION_DIR = "/home/ebr/data/PTHA2020_runs_UMA/test"

RUNDIR = "/home/ebr/projects/inundation-emulator/generated/emulator_20250211_121156"
EPOCH_CHECKPOINT = 500

emulator = Emulator(GENERATED_DIR, RUNDIR)

In [ ]:
emulator.topomask.shape

In [ ]:
plt.imshow(emulator.topomask)

In [ ]:
from datetime import datetime

# Create output folder
predictions_dir = os.path.join(emulator.rundir, "predictions")
timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
output_dir = os.path.join(predictions_dir, emulator.id, f"preds_{timestamp}")
#os.makedirs(output_dir, exist_ok=True)

emulator.predict(
    scenarios_file=TRAIN_SCENARIOS,
    input_dir=TRAIN_DIR,
    output_dir=output_dir
)

In [ ]:
from datetime import datetime

scenarios_file = TRAIN_SCENARIOS
input_dir = TRAIN_DIR
predictions_dir = "/home/ebr/projects/inundation-emulator/generated/emulator_20250211_121156/predictions"
batch_size = 10

# Create output folder
timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
output_dir = os.path.join(predictions_dir, emulator.id, f"preds_{timestamp}")
os.makedirs(output_dir, exist_ok=True)

with open(scenarios_file, 'r') as file:
    nr_of_scenarios = sum(1 for line in file if line.strip())
    print(f"Number of scenarios for prediction: {nr_of_scenarios}")

reader = DataReader(
        rundir=emulator.rundir, 
        scenarios_file=scenarios_file,
        datadir=input_dir,
        pois=emulator.pois,
        shuffle_on_load=False,
        target=True,
        reload=False
)

output_signature = (
    tf.TensorSpec(shape=(emulator.n_pois, emulator.input_time_steps), dtype=tf.float32),           # eta
    tf.TensorSpec(shape=(reader.topomask.sum()), dtype=tf.float32),         # No flow_depth
    tf.TensorSpec(shape=(), dtype=tf.string)                               # scenario
)

dataset = tf.data.Dataset.from_generator(
        generator=reader.generator,
        output_signature=output_signature
).batch(batch_size)


In [ ]:
import numpy as np

preds = np.zeros((batch_size, *reader.topomask.shape))
flow_depths = np.zeros((batch_size, *reader.topomask.shape))

for eta, flow_depth, scenario_id in dataset:
    # Make predictions
    #preds[:, topomask] = emulator.model(eta, training=False)
    preds[:, emulator.topomask] = emulator.model.predict(eta)
    flow_depths[:,emulator.topomask] = flow_depth
    break


In [ ]:
preds.shape

In [ ]:
scenario_id

In [ ]:
plt.imshow(np.flip(flow_depths[1,:], axis=0))
plt.colorbar()

In [ ]:
plt.imshow(np.flip(preds[1,:], axis=0))
plt.colorbar()

In [ ]:
for scenario, pred in zip(scenario_id, preds):
    print(scenario.numpy().decode('utf-8'))
    print(pred.shape)

In [ ]:
from netCDF4 import Dataset
import json
# Load the stored grid information
with open(emulator.grid_info_file, "r") as f:
    grid_info = json.load(f)

for i in range(10):
    # Create a NetCDF file for each scenario
    scenario_id_str = scenario_id[i].numpy().decode('utf-8')  # Decode scenario_id to string
    prediction_data = preds[i,:]
    pred_file = os.path.join(output_dir, f"{scenario_id_str}_CT_10m_PR.nc")
    
    with Dataset(pred_file, mode='w', format="NETCDF4") as dst:
         # Create dimensions
        for dim_name, dim_size in grid_info["dimensions"].items():
            dst.createDimension(dim_name, dim_size)

        # Create and populate grid variables
        for var_name, var_info in grid_info["variables"].items():
            dst_var = dst.createVariable(var_name, np.dtype(var_info["datatype"]), var_info["dimensions"])

            # Copy attributes
            for attr, value in var_info["attributes"].items():
                dst_var.setncattr(attr, value)

            # Copy data
            dst_var[:] = np.array(var_info["data"])
        
        # Add predicted
        prediction = dst.createVariable("predicted", "f4", ("grid_lat", "grid_lon"))
        prediction.units = "meter"
        prediction.description = "Maximum flow depth."
        prediction[:,:] = prediction_data
        
    print(f"New NetCDF file created: {pred_file}")
        
        

In [ ]:
from pprint import pprint

def inspect_nc_file(file_path):
    # Open the NetCDF file
    dataset = Dataset(file_path, 'r')
    
    # Print general information about the dataset
    print(f"File: {file_path}")
    print(f"Dimensions:")
    pprint(dataset.dimensions)
    
    print(f"\nVariables:")
    pprint(dataset.variables)
    
    print(f"\nGlobal Attributes:")
    pprint(dataset.ncattrs())
    for attr in dataset.ncattrs():
        print(f"{attr}: {getattr(dataset, attr)}")
    
    # Close the dataset
    dataset.close()

# Example usage
#file_path = '/home/ebr/data/PTHA2020_runs_UMA/train_164/1357_E02020N3739E02658N3366-PS-Mur_PYes_Var-M895_E02426N3465_S002_CT_10m.nc'
file_path = '/home/ebr/projects/inundation-emulator/generated/emulator_20250211_121156/predictions/emulator_20250211_121156/preds_20250211_131543/0192_E01548N3896E01854N3659-PS-Str_PYes_Var-M861_E01756N3795_S004_CT_10m_PR.nc'
#file_path = '/home/ebr/data/PTHA2020_runs_UMA/train_164/1357_E02020N3739E02658N3366-PS-Mur_PYes_Var-M895_E02426N3465_S002_ts.nc'
#file_path = "/home/ebr/data/PTHA2020_runs_UMA/Catania/C_CT.grd"
inspect_nc_file(file_path)

In [ ]:

file_path = '/home/ebr/data/PTHA2020_runs_UMA/train_164/1357_E02020N3739E02658N3366-PS-Mur_PYes_Var-M895_E02426N3465_S002_CT_10m.nc'

inspect_nc_file(file_path)

In [ ]:

    # Create an xarray DataArray for the prediction
    data = xr.DataArray(
        preds[i,:],
        dims=topomask.shape,
        name="prediction",
    )

    # Add metadata or attributes if necessary
    data.attrs["scenario"] = scenario_id_str
    data.attrs["description"] = "Model predictions for scenario."
    data.attrs[]

    # Save to NetCDF
    data.to_netcdf(file_path)
    print(f"Saved predictions for scenario '{scenario_id_str}' to {file_path}")